In [55]:
import requests
import zipfile
import time
import os
import unicodedata
from urllib.parse import urljoin
from io import BytesIO
from bs4 import BeautifulSoup

In [53]:
# nastavit na False u serveru, ktere hlasi chybu ssl
sslverify = True
urlRoot = "https://volby.cz/opendata/"
opendatafile = "opendata.htm"
# davky="vysledky_okrsky?davka="
# urlRootDavky = "https://volby.cz/pls/"

In [56]:
opendataUrl = urljoin(urlRoot,opendatafile)
# print (opendataUrl)
response = requests.get(opendataUrl)

https://volby.cz/opendata/opendata.htm


In [4]:
def getUrlsVolby(soup):
    volbyList = []
    for volby in soup.table.find_all("tr"):
        volbyData = volby.find("td").text
        volbyData = unicodedata.normalize('NFKD',volbyData)
        #print(volbyData)
        #print(volba)
        volbyUrls = volby.find_all("a")
        for i in volbyUrls:
            volba = []
            volba.append(volbyData)
            url = i.attrs["href"]
            # print(url)
            volbyId, volbyFileurl = url.split("/")
            volba.append(volbyId)
            volba.append(volbyFileurl)
            volbyList.append(volba)
            # print(volba)
            # print()
    return(volbyList)

In [5]:
if response.status_code == 200:
    HomeSoup = BeautifulSoup(response.content, 'html.parser')
    volbyUrls = getUrlsVolby(HomeSoup)
else:
    print("chyba zpracování", opendataUrl)

In [7]:
def zpracujOdata(url):
    odkazy = []
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    for i in soup.find_all("li"):
        for a in i.find_all("a"):
            odkaz = a.attrs["href"]
            if "http" not in odkaz:
                if "_opendata.htm" in odkaz:
                    odkaz = odkaz.replace("..","/opendata")
                    odkaz = urljoin("https://volby.cz", odkaz)
                    odkazy.append(odkaz)
    return(odkazy)


volbaOpendataHome = []
for i in volbyUrls:
    volbaUrl = urlRoot + i[1] + "/" + i[2]
    time.sleep(0.3)
    if "odata" in volbaUrl or "_seznam" in volbaUrl:
        odkazy = zpracujOdata(volbaUrl)
        for odkaz in odkazy:
            volbaOpendataHome.append(odkaz)
    else:
        volbaOpendataHome.append(volbaUrl)

In [ ]:
sorted(volbaOpendataHome, reverse=True)

In [57]:
def getVolbySoup(home, sleeptime):
    responses = {}
    for i in home:
        volby = i.split("/")[4]
        responses[volby] = requests.get(i)
        time.sleep(sleeptime)
    return (responses)


volbySoupsDict = getVolbySoup(volbaOpendataHome, 2)

In [59]:
volbySoupsDict

{'prez2013': <Response [200]>,
 'prez2018': <Response [200]>,
 'ps2006': <Response [200]>,
 'ps2010': <Response [200]>,
 'ps2013': <Response [200]>,
 'ps2017nss': <Response [200]>,
 'ps2017': <Response [200]>,
 'ps2021': <Response [200]>,
 'se2008': <Response [200]>,
 'se2010': <Response [200]>,
 'se2011': <Response [200]>,
 'se2012': <Response [200]>,
 'se2014leden': <Response [200]>,
 'se2014zari': <Response [200]>,
 'se2014': <Response [200]>,
 'se2016': <Response [200]>,
 'se2017': <Response [200]>,
 'se2018leden': <Response [200]>,
 'se2018kveten': <Response [200]>,
 'se2018': <Response [200]>,
 'se2019duben': <Response [200]>,
 'se2020cerven': <Response [200]>,
 'se2020': <Response [200]>,
 'se2022': <Response [200]>,
 'kz2008': <Response [200]>,
 'kz2012': <Response [200]>,
 'kz2016': <Response [200]>,
 'kz2020': <Response [200]>,
 'kv2006': <Response [200]>,
 'kv2010': <Response [200]>,
 'kv2014': <Response [200]>,
 'kv2018': <Response [200]>,
 'kv2022': <Response [200]>,
 'ep2

In [65]:
for volba, response in volbySoupsDict.items():
    if not os.path.exists(volba):
        os.mkdir(volba)
    url = response.url
    soup = BeautifulSoup(response.content, 'html.parser')
    for a in soup.find_all('a'):
        odkaz = a['href']
        #print(odkaz)
        if '.zip' in odkaz:
            zipUrl = urljoin(response.url, odkaz)
            dataType = odkaz.split('.')[0]
            path = os.path.join(volba,dataType)
            r = requests.get(zipUrl)
            print("processing", path, r.status_code, "\r")
            if r.status_code == 200:
                z = zipfile.ZipFile(BytesIO(r.content))
                z.extractall(path)
            time.sleep(1)
    time.sleep(1)

        
    # print(volba)
    # print(response)
    # for a in volbySoupsD[i].find_all('a'):
    #     print(a['href'])
    #     requests.get(i)
# print(volbySoupsD['prez2013'].prettify())

processing prez2013/PREZ2013reg20130126 200 
processing prez2013/PREZ2013reg20130126_csv 200 
processing prez2013/PREZ2013regXSD 404 
processing prez2013/PREZ2013data20130126 200 
processing prez2013/PREZ2013data20130126_xlsx 200 
processing prez2013/PREZ2013data20130126_csv 200 
processing prez2013/PREZ2013_datovavetaXSD 404 
processing prez2013/PREZ2013ciselniky20130126 200 
processing prez2013/PREZ2013ciselniky20130126_csv 200 
processing prez2013/PREZ2013ciselnikyXSD 404 
processing prez2018/PREZ2018reg20180127 200 
processing prez2018/PREZ2018reg20180127_csv 200 
processing prez2018/PREZ2018regXSD 200 
processing prez2018/PREZ2018data20180127_xml_kolo2 200 
processing prez2018/PREZ2018data20180127_xlsx_kolo2 200 
processing prez2018/PREZ2018data20180127_csv_kolo2 200 
processing prez2018/PREZ2018_datovavetaXSD 200 
processing prez2018/PREZ2018ciselniky20171215 200 
processing prez2018/PREZ2018ciselniky20171215_csv 200 
processing prez2018/PREZ2018ciselnikyXSD 200 
processing ps200

In [ ]:
volby = "prez2018"
ciselniky = "PREZ2018ciselniky20171215_csv.zip"
registry = "PREZ2018reg20180127_csv.zip"
okrskovaData = "PREZ2018data20180127_csv_kolo2.zip"
urlOpendata = urlRoot+volby+"/"+volby+"_"+opendatafileHome
urlDavky = urlRootDavky+volby+"/"+davky
urlCiselniky = urlRoot+volby+"/"+ciselniky
urlRegistry = urlRoot+volby+"/"+registry
urlOkrskovaData = urlRoot+volby+"/"+okrskovaData
davky = True
kola = True

In [ ]:
if not os.path.exists(volby):
    os.mkdir(volby)

In [ ]:
def get_posledni_davka(urlDavky):
    posledni_davka = requests.get(url_davky,verify=sslverify)
    root = ET.fromstring(posledni_davka.text)
    #for child in root:
    #    print (child.tag,child.attrib)
    davka = root[0].attrib
    davky_pocet = int(davka.get("PORADI_DAVKY")) + 1
    return(davky_pocet)

In [ ]:
davky_pocet_kolo1 = get_posledni_davka(url_davky+"&kolo=1")
davky_pocet_kolo2 = get_posledni_davka(url_davky+"&kolo=2")
print(davky_pocet_kolo1,davky_pocet_kolo2)

In [ ]:
def get_data_voleb (volby,davky,kola,ciselniky,registry,okrskovaData)

In [ ]:
def get_davky(kolo,davky_pocet):
    davkydir = os.path.join(volby,"davky"+kolo)
    dir_exists = os.path.exists(davkydir)
    if not dir_exists:
        os.mkdir(davkydir)
    for i in range(1,davky_pocet):
        url=url_davky+str(i)+"&kolo="+kolo
        response=requests.get(url,verify=sslverify)
        time.sleep(1)
        print("davka",i,response.status_code,"\r")
        if i<10:
            davkaurl=os.path.join(davkydir,"davka_0"+str(i)+".xml")
        else:
            davkaurl=os.path.join(davkydir,"davka_"+str(i)+".xml")
        with open(davkaurl,"wb") as file:
            file.write(response.content)
get_davky("1",davky_pocet_kolo1)
get_davky("2",davky_pocet_kolo2) 

In [ ]:
def get_cisregokr():
    for url in [url_ciselniky,url_registry,url_okrskova_data]:
        response = requests.get(url,verify=sslverify)
        zipfile.ZipFile(BytesIO(response.content)).extractall(path=volby)
get_cisregokr()

In [ ]:
response = requests.get(url_ciselniky)

In [ ]:
response.url.split("/")[-1]